<a href="https://colab.research.google.com/github/po3rin/python_playground/blob/master/torchnlp/pytorch_nlp_ch3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!/usr/bin/python
# -*- coding: sjis -*-

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as pn

import pickle

In [2]:
###################

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

##################

with open('drive/MyDrive/resources/dic.pkl','br') as f:
    dic = pickle.load(f)

print(dic['犬'])


labels = {'¼': 0, '': 1, '`e': 2,
     '®': 3, 'âL': 4, '®': 5, 'ã¼': 6,
     'Úö«': 7, '': 8, '`ó': 9, 'L': 10,
     'AÌ': 11, 'Úª«': 12, 'Ú±': 13,
     '´®': 14, 'ó': 15}



4373


In [3]:
# Data setting

with open('drive/MyDrive/resources/xtrain.pkl','br') as f:
    xtrain = pickle.load(f)

with open('drive/MyDrive/resources/ytrain.pkl','br') as f:
    ytrain = pickle.load(f)

print(xtrain[6])
print(ytrain[6])

[74, 75, 2, 60, 76, 62, 5, 6]
[9, 0, 1, 8, 5, 7, 3, 4]


In [4]:
# Define model

class MyLSTM(nn.Module):
    def __init__(self, vocsize, posn, hdim):
        super(MyLSTM, self).__init__()
        self.embd = nn.Embedding(vocsize, hdim)
        self.lstm = nn.LSTM(input_size=hdim, hidden_size=hdim)
        self.ln   = nn.Linear(hdim, posn)
    def forward(self, x):
        x = self.embd(x)
        lo, (hn, cn) = self.lstm(x)
        out = self.ln(lo)
        return out

# model generate, optimizer and criterion setting

net = MyLSTM(len(dic)+1, len(labels), 100).to(device)
optimizer = optim.SGD(net.parameters(),lr=0.01)
criterion = nn.CrossEntropyLoss()



In [5]:
for ep in range(1,11):
    loss1K = 0.0
    for i in range(len(xtrain)):
        x = [ xtrain[i] ]
        x = torch.LongTensor(x).to(device)
        output = net(x)
        y = torch.LongTensor( ytrain[i] ).to(device)
        loss = criterion(output[0],y)
        if (i % 1000 == 0):
            print(i, loss1K)
            loss1K = loss.item()
        else:
            loss1K += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    outfile = "lstm0-" + str(ep) + ".model"
    torch.save(net.state_dict(),outfile)


0 0.0
1000 2178.064115524292
2000 1537.2481379508972
3000 1291.0816811919212
4000 1174.923682719469
5000 1093.4489098787308
6000 1029.961230263114
7000 990.7379141747952
8000 950.5596088021994
9000 921.3356041461229
10000 848.3543688505888
11000 845.4519156888127
12000 798.1086546033621
13000 786.9390351306647
14000 751.5163073092699
15000 750.793984785676
16000 711.1174330413342
17000 707.9378633275628
18000 689.5111912116408
19000 669.317695170641
20000 655.2874946370721
21000 632.0807860717177
22000 647.7318616583943
23000 620.3700424954295
24000 631.5447712093592
25000 599.5526193664409
26000 583.1323059424758
27000 592.6080900616944
28000 576.1701840274036
29000 568.9580456409603
30000 550.9118364006281
31000 529.4565925532952
32000 525.5572956914548
33000 510.57676208764315
34000 528.1118008457124
35000 513.687133602798
36000 511.7240861132741
37000 510.6159492107108
38000 483.7604082301259
39000 478.00789900682867
40000 493.25854399614036
41000 490.8724740575999
42000 488.233562

In [12]:

with open('drive/MyDrive/resources/dic.pkl','br') as f:
    dic = pickle.load(f)

with open('drive/MyDrive/resources/label.pkl','br') as f:
    lab = pickle.load(f)

# Data setting

with open('drive/MyDrive/resources/xtest.pkl','br') as f:
    xtest = pickle.load(f)

with open('drive/MyDrive/resources/ytest.pkl','br') as f:
    ytest = pickle.load(f)


# model generate, optimizer and criterion setting

net = MyLSTM(len(dic)+1, len(lab), 100).to(device)
net.load_state_dict(torch.load("lstm0-10.model"))

# Eval

real_data_num = 0  # f[^ÌÂ
net.eval()
with torch.no_grad():
    ok = 0  # ok Í³ð
    for i in range(len(xtest)):
        real_data_num += len(xtest[i])
        x = [ xtest[i] ]
        x = torch.LongTensor(x).to(device)
        output = net(x)
        ans = torch.argmax(output[0],dim=1)
        y = torch.LongTensor(ytest[i]).to(device)
        ok += torch.sum(ans == y).item()
    print(ok, real_data_num, ok/real_data_num)


25446 27271 0.9330790950093506
